In [3]:
import cartopy.crs as ccrs
import datetime
import geopandas as gpd
import matplotlib.font_manager
import warnings

from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, Band, Label, Range1d, BasicTickFormatter, LabelSet, DatetimeTickFormatter, CustomJS, Select, DataRange1d, CheckboxGroup, Span, LinearColorMapper, ColorBar, GeoJSONDataSource, Slider, Whisker, Title, PrintfTickFormatter, NumeralTickFormatter
from bokeh.models.tools import HoverTool
from bokeh.transform import linear_cmap, factor_cmap
from bokeh.palettes import Spectral6
from bokeh.layouts import layout, column, row
from bokeh.io import save
from math import pi
from tqdm import tqdm

#extender o notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:99.5% !important;} </style>"))

warnings.simplefilter(action="ignore", category=FutureWarning)
%config Completer.use_jedi = False

In [5]:
population            = pd.read_csv('data/pop_estimativas_1992-2019.csv',sep=';', skiprows=3,skipfooter=12, engine='python')
population['codibge'] = population['Município'].str.extract(r'([0-9]+)(.*)')[0]
population['name']    = population['Município'].str.extract(r'([0-9]+)(.*)')[1]
population['name']    = population['name'].str[1:].values
del population['Município']

population = population[list(population.columns[-2:])+[population.columns[-3]]].copy(deep=True)
population['population'] = population['2018']
del population['2018']

parana_map     = gpd.read_file('data/municipios_pr_2017_UTM_SAD69_original.shp')
parana_map_low = gpd.read_file('data/municipios_pr_2017_UTM_SAD69.shp')
del parana_map['geometry']

parana_map['geometry'] = parana_map_low['geometry']
del parana_map_low

parana_map.codibge = parana_map.codibge.str[:-1]
parana_map         = pd.merge(
    parana_map,
    population,
    on=['codibge']
)
parana_map = parana_map[[
    'nome', 
    'name', 
    'codibge', 
    'population', 
    'geometry'
]]
parana_map.rename(columns = {
    'name': 'nome_acento', 
})

df            = pd.read_csv('data/auxilio_emergencial.csv')
parana_codes  = parana_map.codibge.unique()
df['codibge'] = [str(code)[:6] for code in df.codigo_ibge_municipio]
df_pr         = df[(df.codibge.isin(parana_codes)) | (df.uf == 'PR')].copy(deep=True)
df_pr.to_pickle('data/auxilio_pr.pkl')
qtd_auxilio   = {
    code: len(df_pr[df_pr.codibge==code]) for code 
    in df_pr.codibge.unique()
}
parana_map['auxilio_emergencial'] = [qtd_auxilio[code] for code in parana_map.codibge]
parana_map.to_pickle('data/auxilio_emergencial_map.pkl')

In [10]:
parana_map = pd.read_pickle('data/auxilio_emergencial_map.pkl')
parana_map['porcentagem_beneficiada'] = 100*parana_map['auxilio_emergencial']/parana_map['population']
df = parana_map.copy(deep=True)

In [16]:
filename           = 'mapa_auxilio'
ultima_atualização = '05/06/2020'

geosource = GeoJSONDataSource(geojson=parana_map.to_json())

fig = figure(
    plot_height = 750,
    plot_width = 950, 
    tools='', 
    sizing_mode="scale_width"
)

fig.xgrid.grid_line_color = None
fig.ygrid.grid_line_color = None 

max_ = np.nanmax(df['porcentagem_beneficiada'])

palette = sns.color_palette("Greens", 2000).as_hex()[400:1700]
color_mapper = LinearColorMapper(
    palette = palette,
    low = 0,
    high = max_
)
patches = fig.patches(
    'xs',
    'ys', 
    source = geosource,
    line_color = '#525252',
    fill_color = {'field': 'porcentagem_beneficiada', 'transform': color_mapper},
    line_width = 0.25,
    fill_alpha = 1
)

fig.add_tools(
    HoverTool(renderers = [patches],
              tooltips  = [
                  ('Cidade','@nome_acento'),
                  ('Porcentagem beneficiada','@porcentagem_beneficiada{1,11.2f}'+'%'),
                  ('Número de concessões','@auxilio_emergencial{,}'),
                  ('População','@population{,} (2018)')
              ],
              formatters = {'@date': 'datetime'},
             )
)
color_bar = ColorBar(
    color_mapper   = color_mapper,
    label_standoff = 8,
    width  = 300, 
    height = 10,
    border_line_color = None,
    location = (0,0),
    orientation = 'horizontal',
    title = '% da população beneficiada pelo auxílio emergencial'
)    

color_bar.title_text_font_style = 'normal'
color_bar.formatter  = NumeralTickFormatter(format="0,0")
fig.toolbar.logo     = None
fig.toolbar_location = None
fig.add_layout(color_bar, 'below')    
fig.axis.visible = False
fig.add_layout(Title(text=f"Última atualização: {ultima_atualização}", align="right"), "below")
    
output_file(f"html/{filename}.html")
save(fig, f"html/{filename}.html")

'/home/andre/github/auxilio_emergencial/html/mapa_auxilio.html'